In [ ]:

import numpy as np
import importlib, os, datetime, sys
sys.path.append(os.path.expanduser('~/source'))

from sus.protocol_designer import System, Protocol, Potential, Compound_Protocol
from sus.protocol_designer.protocol import sequential_protocol
from IPython import display
from IPython.display import HTML, Image
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation, PillowWriter
import matplotlib.pyplot as plt

from quick_sim import setup_sim

# self define function

In [ ]:
from numba import njit

@njit
def binary_partition(positions, boundary=0):
    '''
    takes a set of position coordinates and sets each value to either 0 or 1 depending on if it is below or above the boundary
    '''
    return (np.sign(positions-boundary)+1)/2


def separate_by_state_in_full_form(state, **kwargs):
    bool_array_0 = binary_partition(state[:, 0, 0]) == 0
    bool_array_1 = binary_partition(state[:, 0, 0]) == 1.

    return {
        "0": bool_array_0, "1": bool_array_1,
    }



In [ ]:

def customizedProtocol(initial_values_dict, protocol_list, normalized = False):
    protocol_key_array = initial_values_dict.keys()
    protocol_parameter_dict = {key: [value] for key, value in initial_values_dict.items()}
    protocol_parameter_dict["t"] = [0.0]


    for item in protocol_list:
        # add the duration to the time entry of the protocol_parameter_dict
        protocol_parameter_dict["t"].append(protocol_parameter_dict["t"][-1] + item["duration"])

        for key in protocol_key_array:
            if key in item.keys(): # to check which key is present in the protocol_list_item.
                protocol_parameter_dict[key].append(item[key])
            else:
                protocol_parameter_dict[key].append(protocol_parameter_dict[key][-1])

    if normalized:
        protocol_parameter_dict["t"] = np.array(protocol_parameter_dict["t"])/ np.max(protocol_parameter_dict["t"])

    return protocol_parameter_dict


In [ ]:
def create_system(protocol_parameter_dict, domain = None, modifiedFunction = None):
    """
    This function is used to produce the storage and computation protocol

    input:
    1. comp_protocol_parameter_dict:
    - a dictionary contains the an array of time, which represents the time point at which the protocol is changed
    - the key is the name of the parameter
    - for parameters, they are arrays containing the value of the parameter at the particular time point

    2. modifiedFunction:
    - To modify the simple linear parametrization of the value
    - You can use more complex method to get the values such as calculate the value of phi_1x based on the value of other parameters
    
    output:
    1. comp_prototocl: the protocol for the computation system
    2. storage_protocol: : the protocol for the equilibrium system
    """
    protocol_key = [x for x in computation_protocol_parameter_dict.keys() if x != "t"]

    # storage protocol, just take the first and last element of each term in the protocol key from the dict to form the storage protocol
    storage_t = (protocol_parameter_dict["t"][0], protocol_parameter_dict["t"][-1])
    storage_protocol_parameter_time_series = [np.array([protocol_parameter_dict[key][0], protocol_parameter_dict[key][-1]]) for key in protocol_key]
    storage_protocol_parameter_time_series = np.array(storage_protocol_parameter_time_series)
    storage_protocol = Protocol(storage_t, storage_protocol_parameter_time_series)
    storage_protocol.modifiedFunction = None
    # computation protocol, this part form the time series of the comp_protocol and join them to form Compound_Protocol
    comp_protocol_array = []
    comp_t = protocol_parameter_dict["t"]
    comp_protocol_parameter_time_series = [protocol_parameter_dict[key] for key in protocol_key]
    comp_protocol_parameter_time_series = np.array(comp_protocol_parameter_time_series).T

    if modifiedFunction == None:
        modifiedFunction = [None for _ in range(len(comp_t)-1)]
    
    for i in range(len(comp_t)-1):
        n_th_comp_time_array = (comp_t[i], comp_t[i+1])
        n_th_comp_protocol_parameter_array = np.array([comp_protocol_parameter_time_series[i], comp_protocol_parameter_time_series[i+1]]).T # in the form of array of [(p_n_i, p_n_f)]
        _p = Protocol(n_th_comp_time_array, n_th_comp_protocol_parameter_array)
        
        _p.modifiedFunction = modifiedFunction[i]
        comp_protocol_array.append(_p)
    comp_protocol = Compound_Protocol(comp_protocol_array)
    comp_protocol.protocol_array  = comp_protocol_array

    return storage_protocol, comp_protocol

In [ ]:
def protocol_item(potential_params, duration, name):
    _item = potential_params.copy()
    _item['duration'] = duration
    _item['name'] = name
    return _item

from scipy import optimize
def first_derivative_of_pot_function(circuit_params):
    a = circuit_params["a"]
    b = circuit_params["b"]
    c = circuit_params["c"]
    d = circuit_params["d"]
    def Fcn(x):
        return 4 * a * x**3 + 3 * b * x**2 + 2 * c * x + d
    return Fcn

def first_derivative_of_pot_function(circuit_params):
    beta_1 = circuit_params["beta_1"]
    phi_1xdc = circuit_params["phi_1xdc"]
    
    def Fcn(x):
        return 1/2 * x**2 + beta_1 * np.cos(x) * np.cos(phi_1xdc/2)
    return Fcn


def find_all_critical_points_for_all_potential(circuit_params, guess = [-2, 0, 2]):
    """"
    To find all the critical points, including the saddle, minimum and maximum points, of the potential function
    """
    critical_points = [optimize.fsolve(first_derivative_of_pot_function(circuit_params), _g) for _g in guess]

    # energy_set = [coupled_flux_qubit_non_linear_approx_pot(sol[0], sol[1], _phi_1dcx, _phi_2dcx, _params_at_t) for sol in solution_set]
    print("This is the critical Potential")
    return critical_points
    

In [ ]:
def qfp_1D_dimensionless_potential(x, params):
    beta_1, phi_1xdc = params
    return 1/2 * x**2 + beta_1 * np.cos(x) * np.cos(phi_1xdc/2)

def first_derivative_of_qfp_1D_dimensionless_potential_function(circuit_params):
    beta_1 = circuit_params["beta_1"]
    phi_1xdc = circuit_params["phi_1xdc"]
    
    def Fcn(x):
        return 1/2 * x**2 + beta_1 * np.cos(x) * np.cos(phi_1xdc/2)
    return Fcn


fig, ax = plt.subplots(1, 1, figsize = [5, 5])
initial_parameter_dict = {'beta_1': 2.0, 'phi_1xdc': 0}
guess = [-2, 2]
U_ratio = 0 
max_barrier_height = 20


for i, param_dict in enumerate([initial_parameter_dict]):
    x_array = np.linspace(-4, 4, 100)
    PE_array = qfp_1D_dimensionless_potential(x_array, params = param_dict.values())
    critical_points = [optimize.fsolve(first_derivative_of_qfp_1D_dimensionless_potential_function(initial_parameter_dict), _g) for _g in guess] + [0]
    critical_points_PE = [qfp_1D_dimensionless_potential(x, param_dict.values()) for x in critical_points]
    barrier_height = critical_points_PE[2] - critical_points_PE[0]
    print(critical_points)
    print(f"U_ratio = {max_barrier_height / barrier_height}")
    U_ratio = float(max_barrier_height / barrier_height)

    ax.plot(x_array, PE_array)
    # ax.scatter(critical_points, critical_points_PE)
    ax.set_xlim(-4, 4)
    # ax.set_ylim(1.17, 1.244)

delta_U_unitless = 1.17, 1.244

# circuit parameters

In [ ]:
has_velocity = True

PHI_0 = 2.067833848 * 1e-15
k_B = 1.38e-23
T = 0.5
# T = 7
k_BT = k_B * T

C_factor = 1
L_factor = 1
R_factor = 100
# I_m_factor = 50
# I_m_factor = 15
I_m_factor = 0
time_scale = 1.0


I_p_1, I_p_2 = 5e-6 , 5e-6  # Amp
I_m_1, I_m_2 = 7e-9 * I_m_factor, 7e-9 * I_m_factor                           # Amp
R_1, R_2 = 1 * R_factor, 1 * R_factor                                         # ohm
C_1, C_2 = 500e-15 * C_factor, 500e-15 * C_factor                             # F

L_1, L_2 = 140e-12 * L_factor, 140e-12 * L_factor                             # H 
# L_1, L_2 = 5e-12 * L_factor, 5e-12 * L_factor                             # H 
freq = 1/np.sqrt(C_1 * L_1)
characteristic_time = np.sqrt(C_1 * C_factor * L_1 * L_factor)
print(f"freq = {freq / 1e9:.3f}GHz, characteristic_time  = {characteristic_time * 1e12:.3f}ps")


In [ ]:

m_c = C_1
m_1 = C_1
m_2 = C_2
x_c = PHI_0 / (2 * np.pi)
time_scale_factor = 1
t_c = np.sqrt(L_1 * C_1)
v_c = x_c / t_c


U0_1 = m_c * x_c**2 / t_c**2 / k_BT
U0_2 = m_2 * x_c**2 / t_c**2 / k_BT
kappa_1, kappa_2, kappa_3, kappa_4 = 1/U0_1, 1/U0_1, 1/U0_1, 1/U0_1

lambda_1 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_1  = 1
eta_1    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_1 / 1**2)

lambda_2 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_2  = 1 / (C_2/C_1)
eta_2    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_2 * (R_1 * C_1**2) / (R_2 * C_2**2))

lambda_3 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_3  = 4
eta_3    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_3)

lambda_4 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_4  = 4 / (C_2/C_1)
eta_4    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_4 * (R_1 * C_1**2) / (R_2 * C_2**2))

gamma = 10


beta_1 = 2 * np.pi * L_1 * I_p_1 / PHI_0; 
beta_2 = 2 * np.pi * L_2 * I_p_2 / PHI_0;

beta_1 = 2.3
beta_2 = 2.3

d_beta_1 = 2 * np.pi * L_1 * I_m_1 / PHI_0; 
d_beta_2 = 2 * np.pi * L_2 * I_m_2 / PHI_0;

_lambda = np.array([lambda_1])
_theta  = np.array([theta_1])
_eta  =   np.array([eta_1])

In [ ]:
_lambda

In [ ]:
U0_1 = m_c * x_c**2 / t_c**2 / k_BT
U0_1 = U0_1 * 0.06
U0_1 = U_ratio
print(U0_1)

### parameter setting

In [ ]:
"""
# step 0: modify parameters
- All the parameters are stored in a separate file PARAMETER_INPUT
- You can override some of the parameters here.
"""
params = {}
params['N'] = 1000
params['dt'] = 1/500
params['lambda'] = 1
params['beta'] = 1
params['sim_params'] = [_lambda, _theta, _eta]
params['target_work'] = None
params['applyOffset'] = False
params['measureWorkWithOffset'] = False
params['monitor_work_dist_in_whole_process'] = True # To monitor the work process
params['comment'] = "Experiment 8 (2024/3/17): 4 well, with no compensation for asym, 1/5000"
params['capacitance'] = [C_1]
params['mass'] = np.array([1])
params['v_c'] = x_c/t_c
params['k_BT'] = k_BT
params['U0'] = U0_1
params['as_step'] = np.s_[::10] # the time step to skep for the all_state
params['percentage'] = 1 # For what percentage of the total sample do you want to keep in the output all_state


## define potential

In [ ]:
def simple_1D_potential(x, params):
    a, b, c, d = params
    return U0_1 * (a * x**4 + b * x**3 + c * x**2 + d * x)

def simple_1D_force(x, params):
    a, b, c, d = params
    return -(4 * a * x**3 + 3 * b * x**2 + 2 * c * x + d)


def quadratic_potential(x, params):
    a, b = params
    return U0_1 * 1/2 * a * (x - b)**2


def quadratic_force(x, params):
    a, b = params
    return -a * (x - b)

def qfp_1D_potential(x, params):
    beta_1, phi_1xdc = params
    return U_ratio * (1/2 * x**2 + beta_1 * np.cos(x) * np.cos(phi_1xdc/2))

def qfp_1D_force(x, params):
    beta_1, phi_1xdc = params
    return -(x - beta_1 * np.sin(x) * np.cos(phi_1xdc/2))

potential_used = quadratic_potential
force_used = quadratic_force
number_of_parameter = 2
param_dict = [1, 0]

In [ ]:
U0_1

In [ ]:
x_array = np.linspace(-4, 6, 100)
PE_array = potential_used(x_array, params = param_dict)
PE_array = PE_array - min(PE_array)
plt.plot(x_array, PE_array)


In [ ]:

"""
# step 1: Define initial condition and protocol
"""
manual_domain=[np.array([-5]), np.array([5])]
# initial_parameter_dict = {'a': 10, 'b': 0, 'c': -20, 'd': 0}
initial_parameter_dict = {'a': 1/4, 'b': 0, 'c': 1/2, 'd': 0}
initial_parameter_dict = {'beta_1': 2.0, 'phi_1xdc': 0}
initial_parameter_dict = {'a': 1.0, 'b': 0.0}

# initial_parameter_dict = parameter_dict_2.copy()
"""
# step 2: Define potential
"""
# coupled_fq_pot = Potential(coupled_flux_qubit_pot_with_offset_at_00_xy, coupled_flux_qubit_force, 14, 4,\
#                            default_params = coupled_fq_default_param,  relevant_domain = coupled_fq_domain)

coupled_fq_pot = Potential(potential_used, force_used, number_of_parameter, 1, \
                           default_params = initial_parameter_dict,  relevant_domain = manual_domain)

## define protocol

In [ ]:
# parameter_dict_1 = { "a": 10,     "b":0,     "c": -20,  "d": 0 }

# b_1 = 4
# parameter_dict_2 = { "a": 0,     "b":0,     "c": 10,  "d": -20 }

# b_2 = 1
# parameter_dict_3 = { "a": 2,     "b":-b_2,     "c": -4,  "d": 3.5 * b_2 }

# parameter_dict_4 = { "a": 10,     "b":0,     "c": -20,  "d": 0 }

# fig, ax = plt.subplots(1, 4, figsize = [20, 5])


# for i, param_dict in enumerate([parameter_dict_1, parameter_dict_2, parameter_dict_3, parameter_dict_4]):
#     x_array = np.linspace(-4, 4, 100)
#     PE_array = simple_1D_potential(x_array, params = param_dict.values())
#     critical_points = find_all_critical_points_for_all_potential(param_dict, guess = [-2, 0, 2])
#     critical_points_PE = [simple_1D_potential(x, param_dict.values()) for x in critical_points]


#     ax[i].plot(x_array, PE_array)
#     ax[i].scatter(critical_points, critical_points_PE)
#     ax[i].set_xlim(-4, 4)
#     # ax[i].set_title(critical_points)
#     ax[i].set_ylim(-500, 400)

# # ax[1].scatter(final_state_x, final_state_PE)
# # ax[1].plot(x_array, PE_final_array)
# # ax[1].set_xlim(-4, 4)
# # ax[1].set_ylim(-300, 200)

In [ ]:
zeroDissipation = False
zeroDissipation = True
saveAllStates = True

params['sim_params'] = [_lambda, _theta, _eta]
if zeroDissipation:
    params['sim_params'] = [_lambda * 0, _theta, _eta * 0]

# protocol_list = [
#     # {"duration": 10, "a": 1, "b": 0, "c": -2, "d": 0, "name":"decrease k"}
#     protocol_item(parameter_dict_2, 0.01, "acceleration_protocol"),
#     protocol_item(parameter_dict_2, 0.03, "acceleration_protocol"),
#     protocol_item(parameter_dict_3, 0.01, "decceleration_protocol"),
#     protocol_item(parameter_dict_3, 0.03, "decceleration_protocol"),
#     protocol_item(parameter_dict_4, 0.01, "two well"),
#     protocol_item(parameter_dict_4, 10, "two well")
# ]
protocol_1 = {"a": 1000, "b": 0}

protocol_list = [ protocol_item(protocol_1, 1000, "initial_params") ]

# protocol_list = [ protocol_item(parameter_dict_2, 20, "acceleration_protocol") ]

In [ ]:

# protocol_list = [{'a': 0,
#   'b': 0,
#   'c': 2,
#   'd': -4,
#   'duration': 0.01,
#   'name': 'acceleration_protocol'},
#  {'a': 0,
#   'b': 0,
#   'c': 2,
#   'd': -4,
#   'duration': 0.3,
#   'name': 'acceleration_protocol'},
#  {'a': 2,
#   'b': -1,
#   'c': -4,
#   'd': 3.5,
#   'duration': 0.01,
#   'name': 'decceleration_protocol'},
#  {'a': 2,
#   'b': -1,
#   'c': -4,
#   'd': 3.5,
#   'duration': 0.3,
#   'name': 'decceleration_protocol'},
#  {'a': 10, 'b': 0, 'c': -20, 'd': 0, 'duration': 0.01, 'name': 'two well'},
#  {'a': 10, 'b': 0, 'c': -20, 'd': 0, 'duration': 10, 'name': 'two well'}]

In [ ]:
C_1

## set up cfqr runner

In [ ]:
from edward_tools import potential_runner_1D
from infoenginessims.simprocedures import basic_simprocedures as sp
from infoenginessims.simprocedures import running_measurements as rp
from infoenginessims.simprocedures import trajectory_measurements as tp
from infoenginessims.simprocedures.basic_simprocedures import ReturnFinalState

computation_protocol_parameter_dict = customizedProtocol(initial_parameter_dict, protocol_list)
storage_protocol, comp_protocol = create_system(computation_protocol_parameter_dict, modifiedFunction = None)

cfqrRunner = potential_runner_1D.simRunner(potential=coupled_fq_pot, params=params, potential_default_param=initial_parameter_dict, storage_protocol = storage_protocol, computation_protocol = comp_protocol , measure_all_states = True, protocol_list = protocol_list, has_velocity = True)
cfqrRunner.initialize_sim()

cfqrRunner.eq_system.k_BT = k_BT
cfqrRunner.eq_system.U0 = U0_1
cfqrRunner.eq_system.capacitance = [C_1]
cfqrRunner.eq_system.v_c = v_c

cfqrRunner.system.k_BT = k_BT
cfqrRunner.system.U0 = U0_1
cfqrRunner.system.capacitance = [C_1]
cfqrRunner.system.v_c = v_c


cfqrRunner.procs =  [ sp.ReturnFinalState(), sp.MeasureAllState(), sp.MeasureWorkDone(rp.get_dW), rp.MeasureAllValue(rp.get_dW, 'all_W')]


## init state

In [ ]:
init_state_generated = cfqrRunner.eq_system.eq_state(cfqrRunner.params['N'], t=0, beta=cfqrRunner.params['beta'], manual_domain = manual_domain)
# print(f"The mean KE of all particle = {np.mean(cfqrRunner.system.get_kinetic_energy(init_state)):.3g}")

In [ ]:
params["N"]

In [ ]:
self_defined_init_state = np.array([[[-2, 0]]]*100)
gaussian_init_state = np.zeros([1000, 1, 2])
gaussian_init_state[:500, 0, 0] = np.random.normal(-2, 0.01, 500)
gaussian_init_state[:500, 0, 1] = np.random.normal(0, 0.1, 500)
gaussian_init_state[500:, 0, 0] = np.random.normal(2, 0.01, 500)
gaussian_init_state[500:, 0, 1] = np.random.normal(0, 0.1, 500)

loaded_init_state = np.load("saved_init_state/1D_toy_model_init_state.npy")

init_state_used = gaussian_init_state
# init_state_used = self_defined_init_state
# init_state_used = loaded_init_state
# init_state_used = init_state_generated

index_0 = separate_by_state_in_full_form(init_state_used)['0']
index_1 = separate_by_state_in_full_form(init_state_used)['1']

In [ ]:
x_array = np.linspace(-4, 4, 100)
PE_init_array = potential_used(x_array, params = initial_parameter_dict.values())
PE_final_array = potential_used(x_array, params = initial_parameter_dict.values())


In [ ]:
initial_state_x_0  = init_state_used[index_0, 0, 0]
initial_state_PE_0 = cfqrRunner.system.get_potential(init_state_used[index_0], 0)

initial_state_x_1  = init_state_used[index_1, 0, 0]
initial_state_PE_1 = cfqrRunner.system.get_potential(init_state_used[index_1], 0)

In [ ]:
initial_state_PE_0

In [ ]:
plt.scatter(initial_state_x_0, initial_state_PE_0, color = "b")
plt.scatter(initial_state_x_1, initial_state_PE_1, color = "r")
plt.plot(x_array, PE_init_array - min(PE_init_array))
plt.xlim(-4, 4)
# plt.ylim(-5, 20)a

In [ ]:
cfqrRunner.set_sim_attributes(init_state = init_state_used, manual_domain = manual_domain, axes = None, percentage = 1, as_step = np.s_[::], verbose = True, extra_constraint = None)
# cfqrRunner.ini_state = init_state


# run simulation

In [ ]:
simResult = cfqrRunner.sim.run()

In [ ]:
protocol_list[-1].values()

In [ ]:
final_parmas = list(protocol_list[-1].values())[0:-2]

x_array = np.linspace(-4, 4, 100)
PE_init_array = potential_used(x_array, params = initial_parameter_dict.values())
PE_final_array = potential_used(x_array, params = final_parmas)

final_state = simResult.final_state



initial_state_x_0  = init_state_used[index_0, 0, 0]
initial_state_PE_0 = cfqrRunner.system.get_potential(init_state_used[index_0], 0)

initial_state_x_1  = init_state_used[index_1, 0, 0]
initial_state_PE_1 = cfqrRunner.system.get_potential(init_state_used[index_1], 0)

final_state_x_0 = final_state[index_0, 0, 0]
final_state_PE_0 = cfqrRunner.system.get_potential(final_state[index_0], 10)
final_state_x_1 = final_state[index_1, 0, 0]
final_state_PE_1 = cfqrRunner.system.get_potential(final_state[index_1], 10)


fig, ax = plt.subplots(1, 2, figsize = [10, 5])
ax[0].scatter(initial_state_x_0, initial_state_PE_0, color = "b")
ax[0].scatter(initial_state_x_1, initial_state_PE_1, color = "r")
ax[0].plot(x_array, PE_init_array)
ax[0].set_xlim(-4, 4)
ax[0].set_ylim(0, 400)

ax[1].scatter(final_state_x_0, final_state_PE_0, color = "b")
ax[1].scatter(final_state_x_1, final_state_PE_1, color = "r")
ax[1].plot(x_array, PE_final_array)
ax[1].set_xlim(-4, 4)
ax[1].set_ylim(0, 400)


ax[0].set_xlim(-4, 4)
# ax[0].set_ylim(-1, 20)
ax[1].set_xlim(-4, 4)
# ax[1].set_ylim(-1, 20)


In [ ]:
frame_skip = 100
end_frame = -1
all_state = simResult.all_state['states'][:, :end_frame:frame_skip, ...]
targetTimeArray = cfqrRunner.protocol_all_time_array[cfqrRunner.as_step][:end_frame:frame_skip]
x_array = np.linspace(-4, 4, 100)

fig, ax = plt.subplots(1, 1, figsize=[10, 10])
initial_state_x  = init_state_used[:, 0, 0]
initial_state_PE = cfqrRunner.system.get_potential(init_state_used, 0)
PE_init_array = potential_used(x_array, params = initial_parameter_dict.values())
scatter_plot = ax.scatter(initial_state_x, initial_state_PE)
ax.set_xlim(-4, 4)
ax.set_ylim(0, 300)
PE_plot = ax.plot(x_array, PE_init_array)[0]
# print()

def animate(i):    
    state_i = all_state[:, i, ...]
    t_i = targetTimeArray[i]
    params_i = cfqrRunner.protocol.get_params(t_i)
    PE_array_i = potential_used(x_array, params = params_i)
    PE_i = cfqrRunner.system.get_potential(state_i, t_i)
    ax.set_title(f"time = {t_i:.3g}")
    scatter_plot.set_offsets(np.c_[state_i[..., 0, 0], PE_i])
    PE_plot.set_ydata(PE_array_i)
    
    #         
ani = FuncAnimation(fig, animate, interval=200, frames=len(targetTimeArray))
writer = animation.PillowWriter(fps=5, metadata=dict(artist='Me'), bitrate=1800)
ani.save('scatter.gif', writer=writer)
plt.close()


In [ ]:
Image(filename="scatter.gif")

In [ ]:
targetTimeArray = cfqrRunner.protocol_all_time_array[cfqrRunner.as_step]
all_state = simResult.all_state['states']
mean_x_0 = np.mean(all_state[index_0, :, 0, 0], axis = 0)
std_x_0 = np.std(all_state[index_0, :, 0, 0], axis = 0) * 3
plt.errorbar(targetTimeArray, y = mean_x_0, yerr = std_x_0)

In [ ]:
std_x_0

In [ ]:
plt.hist(cfqrRunner.sim.work_dist_array, bins = 30)
mean_W = np.mean(cfqrRunner.sim.work_dist_array)
plt.title(mean_W)

In [ ]:
all_state = simResult.all_state['states']
targetTimeArray = cfqrRunner.protocol_all_time_array[cfqrRunner.as_step]

s = plt.plot(targetTimeArray, all_state[:, :, 0, 0].T)
plt.xlim(0, 100)

In [ ]:
from scipy.integrate import quad

def f(x):
  return np.sqrt(2)/np.sqrt(1200 - 50 * x**4 - 100 * x**2)

def f_2(x):
  return np.sqrt(2)/np.sqrt(1200 - 50 * x**4 - 100 * x**2)


I, err = quad(f, -2, 2)
print(I)
print(err)

In [ ]:
I, err = quad(f_2, -2, 2)
print(I)
print(err)

In [ ]:
protocol_list

In [ ]:
cfqrRunner.protocol_time_array